In [1]:
## 키 설정 버전


# 모듈 로딩
import cv2
from yolov5facedetector.face_detector import YoloDetector

model = YoloDetector(target_size=1080, gpu=0, min_face=0)  # yolov5n_state_dict.pt 자동 다운 됨

# 경로 url의 비디오 정보를 반환
webcam = cv2.VideoCapture(0)

# 동영상 파일 열기 성공 여부 확인
if not webcam.isOpened():
    print("Could not open webcam")  # 열려있지 않으면 문자열 출력
    exit()

moja = False
# 비디오 매 프레임 처리
while webcam.isOpened():

    status, frame = webcam.read() 

    bboxes, confs, points = model.predict(frame)

    if not status:
        print("Could not read frame")
        exit()

    key = cv2.waitKey(33)

    if key == 26:
        moja = True

    elif key == 24:
        moja = False
    

    for bbox in bboxes[0]:

        (startX, startY)=bbox[0], bbox[1]
        (endX, endY)=bbox[2], bbox[3]
    

        if moja == True:
            face_region = frame[startY:endY, startX:endX]
                
            M = face_region.shape[0]
            N = face_region.shape[1]

            try: 
                face_region = cv2.resize(face_region, None, fx=0.05, fy=0.05, interpolation=cv2.INTER_AREA)
                face_region = cv2.resize(face_region, (N, M), interpolation=cv2.INTER_AREA)
                frame[startY:endY, startX:endX] = face_region

            except:
                pass
        
    # display output
    cv2.imshow("mojaic_video", frame)  # 윈도우 창에 이미지를 띄움

    

    # cv2.waitkey 키 입력 대기 함수
    if cv2.waitKey(1) & 0xFF == ord('q'):  # 'q' 키 입력 받으면 윈도우 창이 종료
        break
    

webcam.release()  # 동영상 파일을 닫고 메모리 해제
cv2.destroyAllWindows()   # 모든 윈도우 창을 닫음

c:\Users\USER\anaconda3\envs\pytorch38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
# 모자이크 키 병합

# 모듈 로딩
import pafy
import cv2
from yolov5facedetector.face_detector import YoloDetector

url = 'wkit_vs\study\humans_pixabay.mp4'

# url = 'https://www.you?v=hoRAV8Wp2UM'   # 잘못된 경로


def mojaic(url, yolo_type='yolov5n',  target_size=1080, gpu=0, min_face=0, codec='DIVX'):
    
    try:
        video = pafy.new(url)
        best = video.getbest(preftype='mp4')
        uurl = best.url

    except Exception as e:
        uurl = ''
        print('오류 =>\n', e)
        cv2.destroyAllWindows()

    model = YoloDetector(yolo_type=yolo_type, target_size=target_size, gpu=gpu, min_face=min_face)  # yolov5n_state_dict.pt 자동 다운 됨(default)

    # 경로 uurl의 비디오 정보를 반환
    webcam = cv2.VideoCapture(uurl)

    # 재생할 파일의 너비와 높이
    fourcc = cv2.VideoWriter_fourcc(*codec)  # 디지털 미디어 포켓 생성
    width = round(webcam.get(cv2.CAP_PROP_FRAME_WIDTH))  # 프레임의 너비
    height = round(webcam.get(cv2.CAP_PROP_FRAME_HEIGHT))  # 프레임의 높이
    fps = webcam.get(cv2.CAP_PROP_FPS)  # 프레임 속도

    out = cv2.VideoWriter('mosaic.avi', fourcc, fps, (width, height))  # 파일명 경로

    # 동영상 파일 열기 성공 여부 확인
    if not webcam.isOpened():
        print("Could not open webcam")  # 열려있지 않으면 문자열 출력
        exit()
    

    moja = False
    # 비디오 매 프레임 처리
    while webcam.isOpened():

      
        status, frame = webcam.read() 

        # frame=cv2.resize(frame, dsize=(0,0), fx=0.7, fy=0.7, interpolation=cv2.INTER_CUBIC)

        # frame=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        bboxes, confs, points = model.predict(frame)

        if not status:
            print("Could not read frame")
            exit()

        key = cv2.waitKey(33)

        if key == 26:  # Ctrl + Z  <- 모자이크 켜짐
            moja = True

        elif key == 24:  # Ctrl + X <- 모자이크 꺼짐
            moja = False


        for bbox in bboxes[0]:

            (startX, startY)=bbox[0], bbox[1]
            (endX, endY)=bbox[2], bbox[3]
        

            # cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)

            # 모자이크 처리
            if moja == True:
                face_region = frame[startY:endY, startX:endX]  # 관심영역(얼굴) 지정
                
                M = face_region.shape[0]
                N = face_region.shape[1]

                try: 
                    face_region = cv2.resize(face_region, None, fx=0.05, fy=0.05, interpolation=cv2.INTER_AREA)
                    face_region = cv2.resize(face_region, (N, M), interpolation=cv2.INTER_AREA)
                    frame[startY:endY, startX:endX] = face_region  # 원본 이미지에 적용

                except:
                    pass

            

        # display output
        # frame=cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        out.write(frame)
        cv2.imshow("Real-time face detection", frame)  # 윈도우 창에 이미지를 띄움

        

        # cv2.waitkey 키 입력 대기 함수
        if cv2.waitKey(1) & 0xFF == ord('q'):  # 'q' 키 입력 받으면 윈도우 창이 종료
            break
        
    out.release()  # 동영상 파일을 닫고 메모리 해제
    webcam.release()  # 동영상 파일을 닫고 메모리 해제
    cv2.destroyAllWindows()   # 모든 윈도우 창을 닫음

# 함수 호출
mojaic(url)

c:\Users\USER\anaconda3\envs\pytorch38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


오류 =>
 Need 11 character video id or the URL of the video. Got //wkit_vs\study\humans_pixabay.mp4
Could not open webcam


: 

In [6]:
import cv2

def video_save(webcam, codec, file_name):

    fourcc = cv2.VideoWriter_fourcc(*codec)  # 디지털 미디어 포켓 생성
    width = round(webcam.get(cv2.CAP_PROP_FRAME_WIDTH))  # 프레임의 너비
    height = round(webcam.get(cv2.CAP_PROP_FRAME_HEIGHT))  # 프레임의 높이
    fps = webcam.get(cv2.CAP_PROP_FPS)  # 프레임 속도

    out = cv2.VideoWriter(file_name, fourcc, fps, (width, height))  # 파일명 경로

    return out